# Appendix B:  Data Processing Pipeline (Code Listing)

## B1. Setup, Imports, and Project Folders

In [ ]:
import pandas as pd
import numpy as np


from datetime import datetime

from pytrends.request import TrendReq
import requests


import matplotlib.pyplot as plt
import seaborn as sns


import os
import warnings
warnings.filterwarnings("ignore") #for clean output and keeps notebook clean
from pathlib import Path

# Project folders (relative to this notebook's working directory)
DATA_RAW = Path("data_raw")
DATA_CLEAN = Path("data_clean")


## B2. Washington, DC; Combine Yearly Files into One Raw CSV

Purpose: merge multiple DC CSV extracts into a single raw file for cleaning. (Includes a quick column preview.)

In [ ]:
dc_files = sorted(DATA_RAW.glob("dc_*.csv"))
print("DC files:", dc_files)

dc_all = pd.concat([pd.read_csv(f) for f in dc_files], ignore_index=True)

dc_all.to_csv(DATA_RAW / "washington_dc_311_raw.csv", index=False)

print("DC combined shape:", dc_all.shape)



In [ ]:
df = pd.read_csv(DATA_RAW / "washington_dc_311_raw.csv", nrows=5) #to check column names
print(df.columns)


## B3. Boston; Merge Yearly Raw Files

Purpose: combine Boston yearly CSVs into a single raw dataset and define output paths.

In [ ]:
import pandas as pd
from pathlib import Path

BOSTON_DIR = Path("data/raw/boston_yearly")
out_raw_path = Path("data/raw/311_boston_raw_2019_2025.csv")
out_clean_path = Path("data/clean/311_boston_2019_2025.csv")

def merge_boston_yearly_csvs(folder: Path) -> pd.DataFrame:
    files = sorted(folder.glob("*.csv"))
    if not files:
        raise FileNotFoundError(f"No CSV files found in {folder.resolve()}")

    parts = []
    for f in files:
        # Read the CSV
        df = pd.read_csv(f)

        # to keep track of which file it came from
        df["source_file"] = f.name

        parts.append(df)

        print(f"Loaded {f.name}: {len(df):,} rows")

    merged = pd.concat(parts, ignore_index=True)
    print(f"\nMerged total: {len(merged):,} rows")
    return merged

# 1) Merge raw yearly files
boston_raw = merge_boston_yearly_csvs(DATA_RAW)

# 2) Save the merged raw (to never have to re-merge)
out_raw_path.parent.mkdir(parents=True, exist_ok=True)
boston_raw.to_csv(out_raw_path, index=False)

# 3) Clean + standardize to your canonical schema
BOSTON_DATE_COL = "open_dt"
BOSTON_TYPE_COL = "type"

boston_clean = boston_raw.copy()
boston_clean["date"] = pd.to_datetime(boston_clean[BOSTON_DATE_COL], errors="coerce")
boston_clean["complaint_type"] = boston_clean[BOSTON_TYPE_COL].astype("string")
boston_clean["city"] = "Boston"

boston_clean = boston_clean[["city", "date", "complaint_type"]].dropna(subset=["date", "complaint_type"])
boston_clean = boston_clean[boston_clean["complaint_type"].str.len() > 0]
boston_clean = boston_clean[(boston_clean["date"] >= "2019-01-01") & (boston_clean["date"] < "2026-01-01")]

# 4) Save clean
out_clean_path.parent.mkdir(parents=True, exist_ok=True)
boston_clean.to_csv(out_clean_path, index=False)

# 5) Sanity check year coverage
print("\nBoston years:", boston_clean["date"].dt.year.value_counts().sort_index().to_dict())

## B4. City-Level Cleaning Examples

Purpose: apply a standardized cleaning function to raw city-level 311 datasets to improve formatting and write cleaned outputs (Seattle and Washington, DC examples).

In [ ]:
dc_raw = pd.read_csv("data_raw/washington_dc_311_raw.csv")  
dc_clean = clean_311(dc_raw, "Washington DC", "ADDDATE", "SERVICECODEDESCRIPTION")
dc_clean.to_csv("data_clean/311_washington_dc_2019_2025.csv", index=False)
print(dc_clean["date"].dt.year.value_counts().sort_index().to_dict())

In [ ]:
import pandas as pd

sea_raw = pd.read_csv("data_raw/seattle_2018_2025_raw.csv", nrows=5)
print(sea_raw.columns.tolist())

In [ ]:
import pandas as pd

sea_raw = pd.read_csv("data_raw/seattle_2018_2025_raw.csv")

sea_clean = clean_311(
    sea_raw,
    city="Seattle",
    date_col="Created Date",
    type_col="Service Request Type"
)

sea_clean.to_csv("data_clean/311_seattle_2019_2025.csv", index=False)

# sanity check
print(sea_clean["date"].dt.year.value_counts().sort_index().to_dict())

In [ ]:
dc = pd.read_csv("data_clean/dc_311_clean.csv")

dc = dc.rename(columns={
    "created_date": "date",
    "service_request_type": "complaint_type"
})

dc = std(dc, "Washington DC")

## B5. Standardize Cleaned City Files and Combine into One 311 Table

Purpose: load cleaned city files, standardize columns, and concatenate into a unified 311 dataframe (`df311`).

In [ ]:

sea = pd.read_csv("data_clean/seattle_311_clean.csv")
bos = pd.read_csv("data_clean/boston_311_clean.csv")
dc  = pd.read_csv("data_clean/dc_311_clean.csv")

def std(df, city_name):
    df = df.copy()
    df["city"] = city_name if "city" not in df.columns else df["city"]
    df["date"] = pd.to_datetime(df["date"], errors="coerce")
    df["complaint_type"] = df["complaint_type"].astype(str)
    return df.dropna(subset=["date"])

sea = std(sea, "Seattle")
bos = std(bos, "Boston")
dc  = std(dc, "Washington DC")

df311 = pd.concat([sea, bos, dc], ignore_index=True)
df311.head()

In [ ]:

# Reload from disk 
sea_raw = pd.read_csv("data_clean/seattle_311_clean.csv")
bos_raw = pd.read_csv("data_clean/boston_311_clean.csv")
dc_raw  = pd.read_csv("data_clean/dc_311_clean.csv")

def std_basic(df, city_name):
    df = df.copy()
    df["city"] = city_name
    df["date"] = pd.to_datetime(df["date"], errors="coerce")
    df["complaint_type"] = df["complaint_type"].astype(str)
    return df.dropna(subset=["date"])[["city","date","complaint_type"]]

# Seattle + Boston are already standardized
sea = std_basic(sea_raw, "Seattle")
bos = std_basic(bos_raw, "Boston")

# DC: detect/rename columns if needed
dc = dc_raw.copy()
dc.columns = [c.strip() for c in dc.columns]

# If DC already has date/complaint_type, this keeps them.
# If not, rename common alternatives:
rename_map = {}
if "date" not in dc.columns:
    for cand in ["created_date", "creation_date", "requested_datetime", "request_date", "open_date"]:
        if cand in dc.columns:
            rename_map[cand] = "date"
            break
if "complaint_type" not in dc.columns:
    for cand in ["sr_type", "service_name", "service_type", "request_type", "type", "category"]:
        if cand in dc.columns:
            rename_map[cand] = "complaint_type"
            break

if rename_map:
    dc = dc.rename(columns=rename_map)

# Now standardize DC
dc = std_basic(dc, "Washington DC")

# Combine
df311 = pd.concat([sea, bos, dc], ignore_index=True)
df311["date"] = pd.to_datetime(df311["date"], errors="coerce", utc=True).dt.tz_convert(None)

print("df311 cities:", df311["city"].value_counts())
print("df311 date range:", df311["date"].min(), "→", df311["date"].max())

df311.head()

## B6. Define Tourism Keywords

Purpose: regex keyword list used to flag tourism-related complaints.

In [ ]:
import re
TOURISM_KEYWORDS = [
    # crowd related
    r"\bnoise\b", r"\bloud\b", r"\bparty\b", r"\bnight\b", r"\bbar\b", r"\bclub\b",
    r"\bmusic\b", r"\bconcert\b", r"\bcrowd\b", r"\bovercrowd(ed|ing)?\b",
    r"\btour\s?group\b",

    # aircraft related
    r"\bairplane\b", r"\baircraft\b", r"\bflight\b",
    r"\bairport\b", r"\bjet\b", r"\bjet\s?noise\b",
    r"\blanding\b", r"\btakeoff\b", r"\bflight\s?path\b",

    # housing related
    r"\bairbnb\b", r"\bshort\s?term\s?rental\b", r"\bvacation\s?rental\b",
    r"\bhotel\b", r"\bmotel\b", r"\bguest\s?house\b",
    r"\bloud\s?guests\b", r"\bparty\s?house\b",

    # public space related
    r"\blitter\b", r"\btrash\b", r"\bgarbage\b", r"\brat(s)?\b",
    r"\bpublic\s?restroom\b", r"\btoilet\b", r"\bsanitation\b",
    r"\boverflow(ing)?\b", r"\bbins?\b",

    # vehicle related
    r"\bparking\b", r"\btraffic\b", r"\bcongestion\b",
    r"\bbus(es)?\b", r"\bmetro\b", r"\bsubway\b", r"\btrain\b",
    r"\btaxi\b", r"\buber\b", r"\blyft\b", r"\brideshare\b",
    r"\btour\s?bus\b",

    # activity related
     r"\bbeach\b", r"\btour\b", r"\btourist(s)?\b", r"\bvisitor(s)?\b",
    r"\bevent\b", r"\bfestival\b", r"\bparade\b",
    r"\bstadium\b", r"\battraction\b", r"\blandmark\b",
]

pattern = re.compile("|".join(TOURISM_KEYWORDS), flags=re.IGNORECASE)

df311["is_tourism_related"] = df311["complaint_type"].apply(lambda s: bool(pattern.search(s)))
df311[["complaint_type", "is_tourism_related"]].head(20)

## B7. Flag Tourism Complaints and Aggregate to Daily and Monthly Metrics

Purpose: apply keyword filter, create daily counts, then aggregate daily → monthly.

In [ ]:
#applied keyword filter
pattern = re.compile("|".join(TOURISM_KEYWORDS), flags=re.IGNORECASE)
df311["is_tourism_related"] = df311["complaint_type"].apply(lambda s: bool(pattern.search(s)))

print(df311["is_tourism_related"].mean())

In [ ]:
# make sure date is datetime (column name)
df311["date"] = pd.to_datetime(df311["date"], errors="coerce")

# optional sanity checks
print(df311["date"].dtype)
print("NaT count:", df311["date"].isna().sum())

# drop rows where date failed to parse
df311 = df311.dropna(subset=["date"])

daily_tourism = (
    df311
    .assign(day=lambda d: d["date"].dt.date)
    .groupby(["city", "day"])
    .agg(
        n_311_calls=("complaint_type", "size"),
        n_tourism_calls=("is_tourism_related", "sum")
    )
    .reset_index()
)

daily_tourism["tourism_call_share"] = daily_tourism["n_tourism_calls"] / daily_tourism["n_311_calls"]
print(sorted(daily_tourism["city"].unique()))

daily_tourism.to_csv(
    "data_clean/tourism_311_daily_sea_bos_dc.csv",
    index=False
)

print("Saved: tourism_311_daily_sea_bos_dc.csv")

In [ ]:
#convert daily to monthly
daily_tourism["day"] = pd.to_datetime(daily_tourism["day"], errors="coerce")

monthly_311 = (
    daily_tourism
    .dropna(subset=["day"])
    .assign(month=lambda d: d["day"].dt.to_period("M").dt.to_timestamp())
    .groupby(["city", "month"])
    .agg(
        n_311_calls=("n_311_calls", "sum"),
        n_tourism_calls=("n_tourism_calls", "sum")
    )
    .reset_index()
)

monthly_311["tourism_call_share"] = monthly_311["n_tourism_calls"] / monthly_311["n_311_calls"]

In [ ]:
sorted(daily_tourism["city"].unique())

## B8. Load Google Trends and Merge with Monthly 311 Metrics

Purpose: load city-month tourism intensity files and merge with monthly 311 burden.

In [ ]:
#load google trends
sea_trends = pd.read_csv("data_clean/seattle_tourism_monthly.csv")
bos_trends = pd.read_csv("data_clean/boston_tourism_monthly.csv")
dc_trends  = pd.read_csv("data_clean/washington_dc_tourism_monthly.csv")

trends = pd.concat([sea_trends, bos_trends, dc_trends], ignore_index=True)
trends["month"] = pd.to_datetime(trends["month"], errors="coerce")
trends = trends.dropna(subset=["month"])

In [ ]:
#merging 311 data and google trends
final_monthly = monthly_311.merge(
    trends,
    on=["city", "month"],
    how="inner"
).sort_values(["city", "month"]).reset_index(drop=True)

final_monthly.to_csv(
    "data_clean/final_monthly_311_plus_trends.csv",
    index=False
)

print("Saved: final_monthly_311_plus_trends.csv")

In [ ]:
print("monthly_311 cities:", sorted(monthly_311["city"].unique()))
print("trends cities:", sorted(trends["city"].unique()))

In [ ]:
#sanity checks
print(final_monthly.groupby("city")["month"].agg(["min", "max", "count"]))
final_monthly.head()

## B.9 Monthly Aggregated Complaint Metrics (Sample Year)

In [ ]:
final_monthly[
    (final_monthly["city"] == "Boston") &
    (final_monthly["month"].dt.year == 2019)
]


In [ ]:
final_monthly[
    (final_monthly["city"] == "Seattle") &
    (final_monthly["month"].dt.year == 2019)
]


In [ ]:
final_monthly[
    (final_monthly["city"] == "Washington DC") &
    (final_monthly["month"].dt.year == 2019)
]


## B10. Visualization and Correlation Checks

Purpose: create diagostic plots and a correlation table.

In [ ]:
import matplotlib.pyplot as plt

for city in final_monthly["city"].unique():
    sub = final_monthly[final_monthly["city"] == city]
    plt.figure()
    plt.scatter(sub["tourism_intensity"], sub["tourism_call_share"])
    plt.title(f"{city}: Tourism Intensity vs Tourism 311 Share")
    plt.xlabel("Tourism Intensity (Google Trends)")
    plt.ylabel("Tourism-related 311 Share")
    plt.show()

In [ ]:
for city in final_monthly["city"].unique():
    sub = final_monthly[final_monthly["city"] == city].sort_values("month")
    plt.figure()
    plt.plot(sub["month"], sub["tourism_call_share"])
    plt.title(f"{city}: Tourism-related 311 Share Over Time")
    plt.xlabel("Month")
    plt.ylabel("Tourism-related 311 Share")
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()

In [ ]:
corr_table = (
    final_monthly
    .groupby("city")[["tourism_intensity", "tourism_call_share"]]
    .corr()
)
corr_table